In [ ]:
import hydromt
import xarray as xr
import numpy as np
from os.path import join, basename
import pandas as pd
# local script skill.py
from skill import skill

In [ ]:
events = {'idai': '20190320', 'eloise':'20210125'}
ddir = r'../../1_data/3_eo_rapid'
fdir = r'../../4_results'
mdir = r'../../3_models/SFINCS'
mdir1 = r'../../3_models/CMF'
hmin=0.25


In [ ]:
# read permanent water mask data
msk, roots = {}, {}
for i, postfix in [(0, '_100m'), (10, '_50m')]:
    root = join(mdir, f'{i:02d}_base{postfix}')
    print(root)
    da_msk_eo = hydromt.open_mfraster(join(root, 'gis', f'dry_*.tif'), concat=True)['dry'].max('dim0')==1
    da_msk_riv = hydromt.open_raster(join(root, 'gis', 'rivmsk.tif'))==1
    da_msk_dep = hydromt.open_raster(join(root, 'gis', 'dep.tif'))<=0
    da_msk = np.logical_or(da_msk_riv, da_msk_dep)
    da_msk = np.logical_or(da_msk, da_msk_eo)
    # da_msk.plot()
    msk[postfix] = da_msk
    roots[postfix] = root

In [ ]:
for event, date in events.items():
    dfs = []
    for postfix in ['_100m', '_50m']:
        da_obs = hydromt.open_raster(join(roots[postfix], 'gis', f'flooding_{date}.tif'))
        da_sim = xr.open_dataarray(join(mdir, f'flddph_{event}{postfix}.nc')).raster.flipud()
        da_skill, da_cm = skill(da_sim, da_obs, msk[postfix], hmin=hmin)
        df_skill = da_skill.reset_coords(drop=True).to_dataframe()
        dfs.append(df_skill)
    df_skill = pd.concat(dfs, axis=0, ignore_index=False)
    df_skill.to_csv(join(mdir, f'flddph_{event}_skill_hmin{hmin}.csv'))
    print(df_skill)
    # break

In [ ]:
# validate CMF
postfix = '_100m'
for event, date in events.items():
    da_obs = hydromt.open_raster(join(roots[postfix], 'gis', f'flooding_{date}.tif'))
    da_sim = xr.open_dataarray(join(mdir1, f'flddph_{event}.nc'))
    da_skill, da_cm = skill(da_sim, da_obs, msk[postfix], hmin=hmin)
    df_skill = da_skill.reset_coords(drop=True).to_dataframe()
    df_skill.to_csv(join(mdir1, f'flddph_{event}_skill_hmin{hmin}.csv'))
    print(df_skill)
    # break

In [ ]:
# plot
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import colors, patheffects
import cartopy.crs as ccrs
from string import ascii_lowercase as abcd

cm_dict = {
    1: ('false neg.', '#dd8452'),
    2: ('false pos.', '#c44e52'),
    3: ('true pos.', '#4c72b0'),
}
levels = [k for k,v in cm_dict.items()] + [4]
colors = [v[1] for k,v in cm_dict.items()]
ticklabs = [v[0] for k,v in cm_dict.items()]
cmap, norm = mpl.colors.from_levels_and_colors(levels, colors)
ticks = np.array(levels[:-1])+np.diff(levels)/2.

In [ ]:
import geopandas as gpd
dep_mask = hydromt.open_raster(join(mdir, '01_rivpowlaw', 'gis', 'dep.tif'))==-9999
riv_mask = hydromt.open_raster(join(mdir, '01_rivpowlaw', 'gis', 'rivmsk.tif'))
riv_mask.raster.set_nodata(0)
gdf_riv = riv_mask.where(~dep_mask, 0).raster.vectorize()
gdf_towns = gpd.read_file(r'../../1_data/towns.geojson').set_index('index').to_crs(gdf_riv.crs)

In [ ]:
hmin=0.25
scores = {'cmf': {}, 'sfx': {}}
res = {'cmf': {}, 'sfx': {}}
for event, date in events.items():
    postfix = '_100m'
    da_obs = hydromt.open_raster(join(roots[postfix], 'gis', f'flooding_{date}.tif')).load()
    da_cmf = xr.open_dataarray(join(mdir1, f'flddph_{event}.nc')).sel(run='00_default_03min')
    scores['cmf'][date],  res['cmf'][date] = skill(da_cmf, da_obs, msk[postfix], hmin=hmin)
    # postfix, run = '_50m', '11_rivpowlaw'
    postfix, run = '_100m', '01_rivpowlaw'
    da_obs = hydromt.open_raster(join(roots[postfix], 'gis', f'flooding_{date}.tif')).load()
    da_sfx = xr.open_dataarray(join(mdir, f'flddph_{event}{postfix}.nc')).sel(run=run).raster.flipud().load()
    scores['sfx'][date],  res['sfx'][date] = skill(da_sfx, da_obs, msk[postfix], hmin=hmin)

In [ ]:

# read crs and utm zone > convert to cartopy
wkt = da_obs.raster.crs.to_wkt()
if "UTM zone " not in wkt:
    raise ValueError("Model CRS UTM zone not found.")
utm_zone = da_obs.raster.crs.to_wkt().split("UTM zone ")[1][:3]
utm = ccrs.UTM(int(utm_zone[:2]), "S" in utm_zone)
extent = np.array(da_obs.raster.box.buffer(100).total_bounds)[[0, 2, 1, 3]]
props = dict( facecolor='w', alpha=0.8)

ann_kwargs = dict(
    xytext=(3, 3),
    textcoords="offset points",
    zorder=4,
    path_effects=[
        patheffects.Stroke(linewidth=3, foreground="w"),
        patheffects.Normal(),
    ],
)

fig, axs = plt.subplots(
    figsize=(9,12),
    nrows=2, ncols=2,
    subplot_kw={'projection': utm},
    sharex = True, sharey=True
)
axs = axs.flatten()
d = {
    'sfx': 'SFINCS',
    'cmf': 'CaMa-Flood',
}

for col, (event, date) in enumerate(events.items()):
    for row, mod in enumerate(['cmf', 'sfx']):
        i = int(row*2 + col)
        ax = axs[i]
        da_cm = res[mod][date].load()
        da_skill = scores[mod][date].load()
        hr, csi, fr = da_skill['H'].item(), da_skill['C'].item(), da_skill['F'].item()

        da_cm.raster.set_crs(gdf_riv.crs)
        da_msk.raster.set_crs(gdf_riv.crs)

        da_msk.where(da_msk).plot(ax=ax, cmap='gray', add_colorbar=False, alpha=0.5)
        cs = da_cm.where(da_cm>0).plot(ax=ax, cmap=cmap, norm=norm, add_colorbar=False)
        # context
        # gdf_riv.boundary.plot(ax=ax, ls='--', lw=0.5, color='k', alpha=0.5)
        # gdf_towns.plot(ax=ax, marker='.', markersize=20, color="k", label='towns / cities', zorder=4)
        for label, grow in gdf_towns.iterrows():
            x, y = grow.geometry.x, grow.geometry.y
            ax.plot(x,y, '.k', markersize=4)
            ax.annotate(f'{label}', xy=(x, y), **ann_kwargs)
        ax.text(0.01, 0.95, abcd[i].upper(), fontsize=14, fontweight='bold', transform=ax.transAxes)
            
        ax.yaxis.set_visible(True)
        ax.xaxis.set_visible(True)
        ax.text(0.8, 0.88, f'C: {csi:.2f}\nH: {hr:.2f}\nF: {fr:.2f}', transform=ax.transAxes, bbox=props)

        if col == 0:
            ax.set_ylabel(f"y coordinate UTM zone {utm_zone} [m]")
        else:
            ax.set_ylabel('')
        
        ax.set_title(f'{d[mod]} - {event} ({date})')
        if i >= len(axs)-2:
            ax.set_xlabel(f"x coordinate UTM zone {utm_zone} [m]")   
        else:
            ax.set_xlabel('')
            
ax.set_xticks(ax.get_xticks()[::2])
ax.set_extent(extent, crs=utm)
fig.subplots_adjust(wspace=0.05, hspace=0.06)

# # Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.93, 0.33, 0.015, 0.3])

# # Draw the colorbar
cbar=fig.colorbar(cs, cax=cbar_ax, orientation='vertical', ticks=ticks)
cbar_ax.set_yticklabels(ticklabs)

plt.savefig(join(fdir, f'validation_hmin25_100m.png'), dpi=500, bbox_inches="tight")

In [ ]:

fig, axs = plt.subplots(
    figsize=(9,12),
    nrows=2, ncols=2,
    subplot_kw={'projection': utm},
    sharex = True, sharey=True
)
axs = axs.flatten()

for col, (event, date) in enumerate(events.items()):
    for row, mod in enumerate(['cmf', 'sfx']):
        i = int(row*2 + col)
        ax = axs[i]
        da_cm = res[mod][date].load()
        da_skill = scores[mod][date].load()
        hr, csi, fr = da_skill['H'].item(), da_skill['C'].item(), da_skill['F'].item()

        da_cm.raster.set_crs(gdf_riv.crs)
        da_msk.raster.set_crs(gdf_riv.crs)

        da_msk.where(da_msk).plot(ax=ax, cmap='gray', add_colorbar=False, alpha=0.5)
        cs = da_cm.where(da_cm>0).plot(ax=ax, cmap=cmap, norm=norm, add_colorbar=False)
        # context
        # gdf_riv.boundary.plot(ax=ax, ls='--', lw=0.5, color='k', alpha=0.5)
        # gdf_towns.plot(ax=ax, marker='.', markersize=20, color="k", label='towns / cities', zorder=4)
        for label, grow in gdf_towns.iterrows():
            x, y = grow.geometry.x, grow.geometry.y
            ax.plot(x,y, '.k', markersize=4)
            ax.annotate(f'{label}', xy=(x, y), **ann_kwargs)
        ax.text(0.01, 0.95, abcd[i].upper(), fontsize=14, fontweight='bold', transform=ax.transAxes)
            
        ax.yaxis.set_visible(True)
        ax.xaxis.set_visible(True)
        ax.text(0.8, 0.88, f'C: {csi:.2f}\nH: {hr:.2f}\nF: {fr:.2f}', transform=ax.transAxes, bbox=props)

        if col == 0:
            ax.set_ylabel(f"y coordinate UTM zone {utm_zone} [m]")
        else:
            ax.set_ylabel('')
        
        ax.set_title(f'{d[mod]} - {event} ({date})')
        if i >= len(axs)-2:
            ax.set_xlabel(f"x coordinate UTM zone {utm_zone} [m]")   
        else:
            ax.set_xlabel('')
            
ax.set_xticks(ax.get_xticks()[::2])
ax.set_extent(extent, crs=utm)
fig.subplots_adjust(wspace=0.05, hspace=0.06)

# # Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.93, 0.33, 0.015, 0.3])

# # Draw the colorbar
cbar=fig.colorbar(cs, cax=cbar_ax, orientation='vertical', ticks=ticks)
cbar_ax.set_yticklabels(ticklabs)

# plt.savefig(join(fdir, f'validation_hmin25_100m.png'), dpi=500, bbox_inches="tight")

In [ ]:
## sensitivity analysis

## barplots
## TODO different rows per event (to scale x axis)
import pandas as pd
cmf_rm = {
    '01_powlaw_06min': '5. spatial res: 6 arcmin',
    # '09_rivpowlaw_lnd0.05': '3. land: manning=0.05 m.s$^{-1/3}$',
    # '06_rivpowlaw_hc0.5': '2c. river: a=0.50, b=0.30',
    '03_powlaw_hp35_03min': '2b. river: a=0.27, b=0.35',
    '02_powlaw_hp25_03min': '2a. river: a=0.27, b=0.25',
    '01_powlaw_03min': '1. default',
}
sfx_rm = {
    '01_rivpowlaw_qh': '4b. forcing: no precip.',
    '01_rivpowlaw_glofas': '4a. forcing: GLOFAS Q',
    '09_rivpowlaw_lnd0.05': '3. land: manning=0.05 m.s$^{-1/3}$',
    # '03_rivgvf': '2d. river: grad. varying flow',
    '06_rivpowlaw_hc0.5': '2c. river: a=0.50, b=0.30',
    '05_rivpowlaw_hp0.35': '2b. river: a=0.27, b=0.35',
    '04_rivpowlaw_hp0.25': '2a. river: a=0.27, b=0.25',
    '01_rivpowlaw': '1. default',
}

sfx_skill, cmf_skill = {}, {}
for event in events:
    df = pd.read_csv(join(mdir, f'flddph_{event}_skill_hmin{hmin}.csv'), index_col=0)
    sfx_skill[event] = df.loc[sfx_rm.keys()].rename(sfx_rm)
    df= pd.read_csv(join(mdir1, f'flddph_{event}_skill_hmin{hmin}.csv'), index_col=0)
    cmf_skill[event] = df.loc[cmf_rm.keys()].rename(cmf_rm)


fig, axs = plt.subplots(2,3, figsize=(15,10), sharey=True)
colors=['darkblue', 'cornflowerblue', 'darkgreen', 'tab:green']
kwargs = dict(color=colors, edgecolor='white', linewidth=1.5, width=0.8, legend=False, zorder=4)
xlims = [
    [[0.60, 0.76], [0.65, 0.91], [0.00,0.21]],
    [[0.25, 0.51], [0.35, 0.85], [0.25,0.51]],
]


for row, event in enumerate(events):
    csi = pd.concat([
        sfx_skill[event]['C'].rename(f'SFINCS'), 
        cmf_skill[event]['C'].rename(f'CaMa-Flood'),
    ], axis=1).sort_index()
    fr = pd.concat([
        sfx_skill[event]['F'].rename(f'SFINCS'), 
        cmf_skill[event]['F'].rename(f'CaMa-Flood'),
    ], axis=1).sort_index()
    hr = pd.concat([
        sfx_skill[event]['H'].rename(f'SFINCS'), 
        cmf_skill[event]['H'].rename(f'CaMa-Flood'),
    ], axis=1).sort_index()

    csi.plot.barh(ax=axs[row, 0], **kwargs)
    hr.plot.barh(ax=axs[row, 1], **kwargs)
    if row == 0:
        fr.plot.barh(ax=axs[row,2], **kwargs).legend(
            bbox_to_anchor=(1.1, 1.1),
            loc="upper right",
            ncol=1,
            fontsize=12

        )
    else:
        fr.plot.barh(ax=axs[row,2], **kwargs)
        axs[row,2].set_xlabel('false alarm rate [-]', fontsize=12)
        axs[row,1].set_xlabel('hit rate [-]', fontsize=12)
        axs[row,0].set_xlabel('critical succes index [-]', fontsize=12)

    for i in range(3):
        axs[row,i].grid(axis='x')
        axs[row,i].tick_params(labelsize=10)
        xmin, xmax = xlims[row][i]
        axs[row,i].set_xlim([xmin, xmax])
        axs[row,i].set_xticks(np.arange(xmin, xmax, 0.05))

    axs[row,1].set_title(event.capitalize(), fontsize=14)
    axs[row,0].set_ylabel('')
    

# fig.tight_layout()
fig.subplots_adjust(wspace=0.05) 
plt.savefig(join(fdir, f'sensitivity.png'), dpi=300, bbox_inches="tight")